Group 19 Final

### Introduction:

### Data Acquisition:

In [2]:
import pandas as pd
%pip install nltk
import nltk
%pip install requests
import requests
#%%
%pip install twint
import twint
import nest_asyncio
import json
import re
from textblob import TextBlob
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nest_asyncio.apply()


Note: you may need to restart the kernel to use updated packages.

  Using cached twint-2.1.20.tar.gz (31 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached aiohttp-3.8.3-cp311-cp311-win_amd64.whl (317 kB)
  Using cached aiodns-3.0.0-py3-none-any.whl (5.0 kB)
  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached cchardet-2.1.7.tar.gz (653 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached elasticsearch-8.5.2-py3-none-any.whl (385 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Using cached aiohttp_socks-0.7.1-py3-none-any.whl (9.3 kB)
  Using cached schedule-1.1.0-py2.py3-none-any.whl (10 kB)
  Using cached geopy-2.3.0-py3-none-any.whl (119 kB)
  Using cached fake_useragent-1.1.1-py3-none-any.whl (50 kB)
  Using cached googletransx-2.4.2.tar.gz (13 kB)
  Preparing metadata (setup.py): started
  Preparing metadat

  error: subprocess-exited-with-error
  
  × Building wheel for pycares (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\pycares
      copying src\pycares\errno.py -> build\lib.win-amd64-cpython-311\pycares
      copying src\pycares\utils.py -> build\lib.win-amd64-cpython-311\pycares
      copying src\pycares\_version.py -> build\lib.win-amd64-cpython-311\pycares
      copying src\pycares\__init__.py -> build\lib.win-amd64-cpython-311\pycares
      copying src\pycares\__main__.py -> build\lib.win-amd64-cpython-311\pycares
      running build_ext
      generating cffi module 'build\\temp.win-amd64-cpython-311\\Release\\_cares.c'
      creating build\temp.win-amd64-cpython-311
      creating build\temp.win-amd64-cpython-311\Release
      building '_cares' extension
    

ModuleNotFoundError: No module named 'twint'

In [ ]:
# c = twint.Config()
# c.Search = 'Nuclear Energy'
# c.Limit = 5000
# c.Store_json = True
# c.Output = 'twit_data.json'
# twint.run.Search(c)

In [ ]:
# c = twint.Config()
# c.Search = 'Nuclear Power'
# c.Limit = 5000
# c.Store_json = True
# c.Output = 'twit_data.json'
# twint.run.Search(c)

### Data Cleaning:

In [ ]:
f = open(r'twit_data.json', 'rb')
data = [json.loads(line) for line in f]
tweet_df = pd.DataFrame(data)
tweet_df = tweet_df[['tweet','link']]
tweet_df = tweet_df.drop_duplicates()
dataremoved = data[~data.iloc[:,0].str.contains('Ukraine')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('ukraine')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Ukrainian')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('ukrainian')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Ukrainians')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('ukrainians')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Zaporizhzhia')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('war')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('russia')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Russia')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('russian')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Russian')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('russians')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Russians')]

In [ ]:
cleaned_tweets = []
words = []
for tweet in dataremoved['tweet']:
    clean = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", tweet)
    clean = re.sub(r"\d", '', clean)
    clean = re.sub(r"'\S+", '', clean)
    clean = clean.replace('.', '').replace(';', '').lower()
    words += re.findall(r"(?:\w+|'|’)+", clean)
    cleaned_tweets.append(clean)

stopwords = nltk.corpus.stopwords.words("english")
standardized = [w for w in words if w not in stopwords]

def wordnet_pos(tag):
    """Map a Brown POS tag to a WordNet POS tag."""

    table = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}

    # Default to a noun.
    # if the key doesn't exist, it will return a wordnet.NOUN
    return table.get(tag[0], wordnet.NOUN)

# all pos_tags for lower words
twit_tags = nltk.pos_tag(standardized)

# obtain tags
tags = [wordnet_pos(x[1]) for x in twit_tags]

# create a new sentence
new_text = " ".join(w for w in standardized)
blob = TextBlob(new_text)
# obtain tags
tags = [wordnet_pos(x[1]) for x in blob.pos_tags]

# finalize the lemmatization
new_text = " ".join(x.lemmatize(t) for x, t in zip(blob.words, tags))
# words after lemmatization
standardized_words = TextBlob(new_text)

(another cleaning method-- on a per tweet basis. we need to sort this out w/ the code above.)

In [5]:
#%%
#all.csv is in the github, it is all of the tweets including the training set tweets we worked on, with the ukraine/russian/war tweets filtered out
data = pd.read_csv('all.csv',header=None)
#%%
data = data.iloc[:,1]
data.columns = ['tweets']

In [10]:
# %% NLP PRE-PROCESSING
import re

cleaned_tweets = []
words = []
for tweet in data:
    clean = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", " ", tweet)
    clean = re.sub(r"\d", '', clean)
    clean = re.sub(r"'\S+", '', clean)
    clean = clean.replace('.', '').replace(';', '').lower()
    words += re.findall(r"(?:\w+|'|’)+", clean)
    cleaned_tweets.append(clean)
    
    
# removing other symbols
corpus = [[re.sub('[^a-zA-Z ]', ' ', document)] for document in cleaned_tweets]
#tokenizing
corpus_tokenized = [nltk.word_tokenize(document[0]) for document in corpus]
# stop words
stopwords = nltk.corpus.stopwords.words("english")
corpus_tokenized = [[word for word in document if word not in stopwords] for document in corpus_tokenized]
#lemmatizing
nltk.download('wordnet')
corpus_lemmatized = [[nltk.WordNetLemmatizer().lemmatize(word) for word in document] for document in corpus_tokenized]
#stitching back together
corpus = [' '.join(document) for document in corpus_lemmatized]

[nltk_data] Downloading package wordnet to C:\Users\L
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### First Model: Preliminary LDA

We will be using the gensim package's LDA model because it seems to have more LDA-specific features such as coherence score calculation.

In [12]:
import numpy as np
import re
import string
#import spacy
%pip install gensim
import gensim
from gensim import corpora
#import pyLDAvis
#import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns

  Using cached gensim-4.2.0.tar.gz (23.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for gensim: started
  Running setup.py install for gensim: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: gensim is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  error: subprocess-exited-with-error
  
  × Running setup.py install for gensim did not run successfully.
  │ exit code: 1
  ╰─> [597 lines of output]
      running install
      c:\Users\L S\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\gensim
      copying gensim\downloader.py -> build\lib.win-amd64-cpython-31

ModuleNotFoundError: No module named 'gensim'

In [ ]:
dictionary = corpora.Dictionary(corpus_lemmatized)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in corpus_lemmatized]

In [ ]:
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=2, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [ ]:
# topic display
posterior = lda_model.print_topics()
two_topic_LDA = pd.DataFrame(posterior)[1]
two_topic_LDA = two_topic_LDA.transpose()
two_topic_LDA.index = ['topic ' + str(i) for i in range(0,2)]
two_topic_LDA.name = 'words'

In [ ]:
# perplexity and coherence scores

print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus_lemmatized, dictionary=dictionary , coherence='u_mass')
if __name__ == "__main__":
    #freeze_support()
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)

We need to identify what number of LDA topics would work best as an input to k-NN.

In [ ]:
#%% GRAPH FUNCTION

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LDA(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='u_mass')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# build list of scores across different topic numbers

model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=corpus_lemmatized, start=2, limit=50, step=1)


In [ ]:
# Show graph
limit=50; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Optimal number = 2??? hahahaha

INTERACTIVE ELEMENT CAN GO HERE

Vectorization

In [ ]:
#%% MATRIX TIME, THIS OUTPUT WILL GO INTO K-NN

tweet_vectors = pd.Series(0)
for i in range(len(doc_term_matrix)):    
    tweet_vectors[i] = lda_model.get_document_topics(doc_term_matrix[i], minimum_probability=0, minimum_phi_value=None, per_word_topics=False)

tweet_vectors_entries = [[tweet_vectors[i][0][1],tweet_vectors[i][1][1]] for i in range(len(tweet_vectors))]

LDA_tweet_frame = pd.DataFrame(tweet_vectors_entries, columns = ['Topic 0','Topic 1'])

### Second Model: K-NN

### Third Model: VADER

### Closing Statements: